### **📄 Title: Dynamic Price Channels for Adaptive Trading**  
**📖 Description:**  
This notebook explores the concept of **Dynamic Price Channels**, which adjust to market volatility using **ATR-based adaptive thresholds**.  

**Why does this matter?**  
- **Fixed thresholds don’t work in all market conditions**.  
- **ATR-based adaptive bands help classify movements more accurately**.  
- **High volatility = wider channel, low volatility = tighter channel**.  

We will implement:  
✔️ **Mathematical formulation of dynamic ATR thresholds**.  
✔️ **A Python-based implementation** for movement classification.  
✔️ **Visualizations to compare fixed vs. adaptive price channels**.  

By the end, you’ll understand **how to build and use a volatility-adjusted price channel** for better market analysis.  

---

## **1️⃣ Introduction**
### **What is a Dynamic Price Channel?**
A **Dynamic Price Channel** is a **volatility-adjusted price range** that adapts to market conditions. Unlike **fixed thresholds**, which remain constant regardless of volatility, **Dynamic Price Channels expand and contract based on recent market activity**.

- **In high volatility**, the channel **widens**, allowing larger price movements to be classified as neutral.  
- **In low volatility**, the channel **shrinks**, making even small price changes significant.  

### **How Does It Work?**
The Dynamic Price Channel is based on **ATR (Average True Range)** and its **dynamic multiplier**, which allows price classifications to adjust based on recent volatility.  

✔️ The **upper and lower bounds** of the channel are defined by:  
\[
\pm (\text{ATR} \times \text{dynamic\_atr\_multiplier})
\]
✔️ Price **must break out of this channel** to be classified as **Up (2) or Down (0)**.  

### **🔹 Understanding the Dynamic Price Channel**
#### **1️⃣ Channel Boundaries**
- The **ATR-based dynamic threshold** acts as a **floating channel** around the price.
- The **width of the channel** expands and contracts **based on recent volatility**.
- If price change **exceeds** the upper/lower bound, it is classified as a **significant movement** (Up/Down).
- If price change **stays within** the channel, it is classified as **Neutral**.

#### **2️⃣ Classification Logic**
- **Inside Channel (Neutral, 1)**
  \[
  -\text{dynamic_volatility_threshold} \leq \text{price_change} \leq \text{dynamic_volatility_threshold}
  \]
- **Breaks Above Channel (Up, 2)**
  \[
  \text{price_change} > \text{dynamic_volatility_threshold}
  \]
- **Breaks Below Channel (Down, 0)**
  \[
  \text{price_change} < -\text{dynamic_volatility_threshold}
  \]
---

---

### **🔹 Visualization: How the Channel Works**
Imagine price moves within this **adaptive ATR-based channel**:

Upper Channel (ATR × Multiplier)
                     ▲
     Price Changes within this range → Neutral (1)
     |-------------------------------------|
     |                                     |
     |    Price Movement (Fluctuations)    |
     |                                     |
     |-------------------------------------|
                     ▼
            Lower Channel (ATR × Multiplier)


- **When volatility is HIGH**, the channel **widens**, requiring larger price changes to classify as Up/Down.
- **When volatility is LOW**, the channel **shrinks**, making even small price changes significant.

---

In [8]:
import pandas as pd
import joblib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

sys.path.append("../..")

import src.indicators.leavitt_indicator as lu

### 🔹 Step 1: Load the Train Dataset
- Ensure the dataset contains the same features used during training.

In [ ]:
# Load train dataset
train_data_path = "../../artifacts/data/processed/train.csv"  # Adjust path as needed
df_train = pd.read_csv(train_data_path)

# Display sample data
print(df_train.head())

   Volume     Open     High      Low    Close      AHMA  Leavitt_Projection  \
0   83787  1.24138  1.25233  1.23849  1.25079  1.247048            1.251170   
1   96966  1.25079  1.25184  1.24092  1.24566  1.248397            1.251928   
2  101428  1.24566  1.24750  1.23625  1.23671  1.246140            1.250534   
3  133358  1.23671  1.24346  1.23138  1.23770  1.243616            1.248112   
4   75446  1.23770  1.24064  1.22460  1.22630  1.238127            1.243717   

   Leavitt_Convolution  LC_Slope  LC_Intercept  ...  Returns_T-10  \
0             1.252753  0.001577      1.248020  ...      0.004399   
1             1.253222  0.001160      1.249742  ...     -0.001463   
2             1.250575 -0.000318      1.251529  ...      0.003298   
3             1.246376 -0.001908      1.252100  ...      0.003067   
4             1.240638 -0.003408      1.250863  ...      0.008863   

   Momentum_T-10  Returns_T-21  Momentum_T-21  Hour  Day_Of_Week  Month  Year  \
0       0.003182     -0.00361

### 🔹 Step 2: Verify Training Dataset

- View it


---

In [6]:
print(df_train.shape)
print(df_train.describe())
print(df_train.columns)

(1167, 29)
              Volume         Open         High          Low        Close  \
count    1167.000000  1167.000000  1167.000000  1167.000000  1167.000000   
mean    75727.157669     1.145632     1.149319     1.141854     1.145445   
std     55250.051016     0.048015     0.047845     0.048178     0.048067   
min         1.000000     1.002060     1.006000     0.995220     1.002060   
25%     37206.500000     1.113360     1.116320     1.110230     1.113235   
50%     67607.000000     1.138370     1.142440     1.135220     1.138070   
75%     98441.500000     1.181045     1.184240     1.177215     1.180985   
max    386009.000000     1.250790     1.255600     1.244830     1.250790   

              AHMA  Leavitt_Projection  Leavitt_Convolution     LC_Slope  \
count  1167.000000         1167.000000          1167.000000  1167.000000   
mean      1.145460            1.145280             1.145087    -0.000193   
std       0.048243            0.048906             0.049277     0.003144   
